**Tugas Kecerdasan Buatan**

---

Oleh : Ruth Elvin - 247056012


In [10]:
import random
import numpy as np

class SchedulingGA:
    def __init__(self, population_size=50, generations=50):
        #inisialisasi parameter untuk algoritma genetika
        self.population_size = population_size
        self.generations = generations
        self.activities = {
            'A': 2,  # durasi A
            'B': 3,  # durasi B
            'C': 2,  # durasi C
            'D': 4   # durasi D
        }
        self.activity_list = ['A', 'B', 'C', 'D']

    def create_individual(self):
        #buat individu dengan waktu mulai acak
        return [random.randint(0, 6) for _ in range(4)]  # Waktu mulai antara 0-6 jam

    def create_population(self):
        #buat populasi dengan sejumlah individu
        return [self.create_individual() for _ in range(self.population_size)]

    def check_constraints(self, individual):
        #check pelanggaran pada individu berdasarkan batasan
        violations = []

        #Constraint 1: A harus sebelum B
        if individual[0] + self.activities['A'] > individual[1]:  #A selesai > B mulai
            violations.append("A harus sebelum B")

        #Constraint 2: C dan D tidak boleh overlap
        c_end = individual[2] + self.activities['C']
        d_end = individual[3] + self.activities['D']
        if not (individual[2] >= d_end or individual[3] >= c_end):  #C dan D overlap
            violations.append("C dan D overlap")

        #Constraint 3: Total durasi <= 8 jam
        end_times = [individual[i] + self.activities[act] for i, act in enumerate(self.activity_list)]
        if max(end_times) > 8:
            violations.append("Melebihi 8 jam")

        return violations

    def calculate_fitness(self, individual):
        #hitung nilai fitness individu
        violations = self.check_constraints(individual) #cek pelanggaram
        return max(0, 10 - len(violations))  #makin sedikit pelanggaran, makin tinggi fitness

    def select_parents(self, population, fitness_scores):
        #pilih parent untuk crossover dengan metode tournament
        tournament_size = 3
        parent1 = max(random.sample(list(enumerate(population)), tournament_size), key=lambda x: fitness_scores[x[0]])[1]
        parent2 = max(random.sample(list(enumerate(population)), tournament_size), key=lambda x: fitness_scores[x[0]])[1]
        return parent1, parent2 #dua parent yang terpilih

    def run(self):
        #run algoritma genetika
        population = self.create_population() #buat populasi awal
        best_solution = None #simpan solusi terbaik
        best_fitness = 0 #simpan nilai fitness terbaik

        print("=== PROSES EVOLUSI ===")
        for generation in range(self.generations):
            fitness_scores = [self.calculate_fitness(ind) for ind in population] #hitung fitness utk tiap individu
            current_best_idx = fitness_scores.index(max(fitness_scores)) #current best index individu
            current_best = population[current_best_idx] #best individu
            current_best_fitness = fitness_scores[current_best_idx] #best fitness individu

            if current_best_fitness > best_fitness: #jika fitness lebih baik dari yang terbaik
                best_fitness = current_best_fitness
                best_solution = current_best.copy() #simpan solusi terbaik
                print(f"\nGenerasi {generation + 1}:")
                print(f"  Solusi Terbaik: {best_solution}, Fitness: {best_fitness}")

            new_population = []
            elite_idx = fitness_scores.index(max(fitness_scores)) #index individu elit
            new_population.append(population[elite_idx])  #masukkan individu elit ke populasi baru

            while len(new_population) < self.population_size:
                parent1, parent2 = self.select_parents(population, fitness_scores) #pilih parent utk crossover

                # Crossover
                point = random.randint(1, len(parent1) - 1) #titik crossover acak
                child1 = parent1[:point] + parent2[point:]
                child2 = parent2[:point] + parent1[point:]

                # Mutasi
                for child in [child1, child2]:
                    if random.random() < 0.1:  # probabilitas mutasi
                        mutation_index = random.randint(0, len(child) - 1) #index yg akan dimutasi
                        child[mutation_index] = random.randint(0, 6) #ubah start time secara random

                new_population.append(child1) #tambahkan child 1 ke populasi baru
                new_population.append(child2) #tambahkan child 2 ke populasi baru

            population = new_population[:self.population_size] #perbarui populasi

        print("\n=== HASIL AKHIR ===")
        print(f"Jadwal Terbaik: {best_solution}")
        print(f"Fitness : {best_fitness}")

        #tampilkan jadwal detail
        self.print_schedule(best_solution)

    def print_schedule(self, individual):
        #display jadwal kegiatan berdasarkan individu
        schedule = []
        for i, activity in enumerate(self.activity_list):
            start_time = individual[i]
            end_time = start_time + self.activities[activity]
            schedule.append((activity, start_time, end_time)) #menyimpan kegiatan beserta start time & end time

        #mengurutkan berdasarkan waktu mulai
        schedule.sort(key=lambda x: x[1])  #urut berdasarkan start_time

        #tampilkan urutan kegiatan
        print("\nUrutan Kegiatan:")
        print(" - ".join(activity for activity, _, _ in schedule))

        # Tampilkan detail jadwal
        print("\nJadwal Detail:")
        for activity, start_time, end_time in schedule:
            print(f"Kegiatan {activity}: Mulai jam {start_time}:00. Selesai jam {end_time}:00")

def main():
    ga = SchedulingGA(population_size=50, generations=50) #inisialisasi algoritma genetika
    ga.run()

if __name__ == "__main__":
    main()


=== PROSES EVOLUSI ===

Generasi 1:
  Solusi Terbaik: [1, 5, 0, 2], Fitness: 10

=== HASIL AKHIR ===
Jadwal Terbaik: [1, 5, 0, 2]
Fitness : 10

Urutan Kegiatan:
C - A - D - B

Jadwal Detail:
Kegiatan C: Mulai jam 0:00. Selesai jam 2:00
Kegiatan A: Mulai jam 1:00. Selesai jam 3:00
Kegiatan D: Mulai jam 2:00. Selesai jam 6:00
Kegiatan B: Mulai jam 5:00. Selesai jam 8:00
